## Clasificador naive bayes
Lo primero que tenemos que hacer es cargar nuestro archivo de datos. Los datos están en formato CSV sin una línea de encabezado ni ninguna comilla. Podemos abrir el archivo con la función de abrir y leer las líneas de datos usando la función de lector en el módulo csv.

También debemos convertir los atributos que se cargaron como cadenas en números para que podamos trabajar con ellos. A continuación se muestra la función loadCsv () para cargar el conjunto de datos de los indios Pima.

## Acerca de Naive Bayes

El algoritmo Naive Bayes es un método intuitivo que utiliza las probabilidades de cada atributo que pertenece a cada clase para hacer una predicción. Es el enfoque de aprendizaje supervisado que se le ocurriría si quisiera modelar un problema de modelado predictivo de forma probabilística.


In [1]:
import csv
import math
import random
def loadCsv():
	lines = csv.reader(open(r'C:\Users\Usuario\Documents\anaconda\correccion\pima-indians-diabetes.csv'))
	dataset = list(lines)
	for i in range(len(dataset)):
		dataset[i] = [float(x) for x in dataset[i]]
	return dataset


A continuación se muestra la función splitDataset () que dividirá un conjunto de datos determinado en una proporción de división determinada.

In [2]:
def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	copy = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(copy))
		trainSet.append(copy.pop(index))
	return [trainSet, copy]

La función asume que el último atributo (-1) es el valor de la clase. La función devuelve un mapa de valores de clase a listas de instancias de datos.

In [3]:
def separateByClass(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated

## Calcular la media
Calcular la media de cada atributo para un valor de clase. La media es la tendencia central media o central de los datos, y la usaremos como la mitad de nuestra distribución gaussiana al calcular las probabilidades.

También debemos calcular la desviación estándar de cada atributo para un valor de clase. La desviación estándar describe la variación de la dispersión de los datos, y la usaremos para caracterizar la dispersión esperada de cada atributo en nuestra distribución gaussiana al calcular las probabilidades.

La desviación estándar se calcula como la raíz cuadrada de la varianza. La varianza se calcula como el promedio de las diferencias al cuadrado para cada valor de atributo de la media. Tenga en cuenta que estamos utilizando el método N-1, que resta 1 del número de valores de atributo al calcular la varianza.

In [4]:
def mean(numbers):
	return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)

## Resumir conjunto de datos
Ahora tenemos las herramientas para resumir un conjunto de datos. Para una lista dada de instancias (para un valor de clase) podemos calcular la media y la desviación estándar para cada atributo.

In [5]:
def summarize(dataset):
	summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	del summaries[-1]
	return summaries

## Resumir atributos por clase
Podemos juntarlo todo separando primero nuestro conjunto de datos de entrenamiento en instancias agrupadas por clase. Luego calcula los resúmenes para cada atributo.

In [6]:
def summarizeByClass(dataset):
	separated = separateByClass(dataset)
	summaries = {}
	for classValue, instances in separated.items():
		summaries[classValue] = summarize(instances)
	return summaries

## Hacer predicción
Ahora estamos listos para hacer predicciones utilizando los resúmenes preparados a partir de nuestros datos de capacitación.
### Calcular la función de densidad de probabilidad gaussiana
Podemos usar una función gaussiana para estimar la probabilidad de un valor de atributo dado, dada la media conocida y la desviación estándar para el atributo estimado a partir de los datos de entrenamiento.

In [7]:
def calculateProbability(x, mean, stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

## Calcular probabilidades de clase
Ahora que podemos calcular la probabilidad de que un atributo pertenezca a una clase, podemos combinar las probabilidades de todos los valores de atributo para una instancia de datos y obtener una probabilidad de que toda la instancia de datos pertenezca a la clase.

In [8]:
def calculateClassProbabilities(summaries, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.items():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, stdev = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev)
	return probabilities

## Haz una predicción
Ahora que podemos calcular la probabilidad de que una instancia de datos pertenezca a cada valor de clase, podemos buscar la mayor probabilidad y devolver la clase asociada.

In [9]:
def predict(summaries, inputVector):
	probabilities = calculateClassProbabilities(summaries, inputVector)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.items():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel

## Hacer predicciones
Finalmente, podemos estimar la precisión del modelo haciendo predicciones para cada instancia de datos en nuestro conjunto de datos de prueba. Los getPredictions () van a hacer esto y devolver una lista de predicciones para cada instancia de prueba.

In [10]:
def getPredictions(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, testSet[i])
		predictions.append(result)
	return predictions

## Obtener precisión
Las predicciones se pueden comparar con los valores de clase en el conjunto de datos de prueba y la precisión de la clasificación se puede calcular como una relación de precisión entre 0 y 100%. El getAccuracy () calculará esta relación de precisión.

In [11]:
def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

In [12]:
def main():
	filename = 'pima-indians-diabetes.csv'
	splitRatio = 0.67
	dataset = loadCsv()
	trainingSet, testSet = splitDataset(dataset, splitRatio)
	print('Dividir {0} filas en tren={1} y prueba={2} filas'.format(len(dataset), len(trainingSet), len(testSet)))
	# prepare model
	summaries = summarizeByClass(trainingSet)
	# test model
	predictions = getPredictions(summaries, testSet)
	accuracy = getAccuracy(testSet, predictions)
	print('Exactitud: {0}%'.format(accuracy))
 
main()

Dividir 768 filas en tren=514 y prueba=254 filas
Exactitud: 73.62204724409449%
